In [2]:
#creating historical db to store data
from quantrocket.history import create_usstock_db
create_usstock_db("usstock-1d",bar_size="1 day",free=True)


{'status': 'successfully created quantrocket.v2.history.usstock-1d.sqlite'}

In [3]:
#collection previous data
from quantrocket.history import collect_history
collect_history("usstock-1d")

{'status': 'the historical data will be collected asynchronously'}

In [2]:
#getting US Stock data
from quantrocket.master import get_securities
securities=get_securities(vendors="usstock",sec_types="STK")
securities.head()
#sid of apple = FIBBG000B9XRY4

,Symbol,Exchange,Country,Currency,SecType,Etf,Timezone,Name,PriceMagnifier,Multiplier,Delisted,DateDelisted,LastTradeDate,RolloverDate
Sid,,,,,,,,,,,,,,
FIBBG000B9XRY4,AAPL,XNAS,US,USD,STK,False,America/New_York,APPLE INC,1,1,False,NaT,NaT,NaT
FIBBG000BFWKC0,MON,XNYS,US,USD,STK,False,America/New_York,MONSANTO CO,1,1,True,2018-06-06,NaT,NaT
FIBBG000BKZB36,HD,XNYS,US,USD,STK,False,America/New_York,HOME DEPOT INC,1,1,False,NaT,NaT,NaT
FIBBG000BMHYD1,JNJ,XNYS,US,USD,STK,False,America/New_York,JOHNSON & JOHNSON,1,1,False,NaT,NaT,NaT
FIBBG000BPH459,MSFT,XNAS,US,USD,STK,False,America/New_York,MICROSOFT CORP,1,1,False,NaT,NaT,NaT


In [ ]:
#creating a universe
from quantrocket.master import create_universe
create_universe("usstock-free",sids=securities.index.to_list())

In [4]:
#pulling price data for the relevant dates ie 2023
from quantrocket import get_prices
prices=get_prices("usstock-1d", universes="usstock-free",start_date="2023-01-01",end_date="2023-12-31",fields=["Close"])
prices.head(5)

Sid               FIBBG000B9XRY4  FIBBG000BKZB36  FIBBG000BMHYD1  \
Field Date                                                         
Close 2023-01-03        124.2163        305.5299        171.6388   
      2023-01-04        125.4975        309.2243        173.5075   
      2023-01-05        124.1666        305.1043        172.2264   
      2023-01-06        128.7352        307.0966        173.6231   
      2023-01-09        129.2616        307.3674        169.1248   

Sid               FIBBG000BPH459  FIBBG000GZQ728  FIBBG00B3T3HD3  
Field Date                                                        
Close 2023-01-03        237.0360        102.0452         43.8958  
      2023-01-04        226.6673        102.3422         44.4374  
      2023-01-05        219.9494        104.6320         44.7919  
      2023-01-06        222.5416        105.8967         45.6485  
      2023-01-09        224.7083        103.9230         49.4985

In [5]:
#extracting apple data using sid
apple_data=prices["FIBBG000B9XRY4"]
print(apple_data)

Field  Date      
Close  2023-01-03    124.2163
       2023-01-04    125.4975
       2023-01-05    124.1666
       2023-01-06    128.7352
       2023-01-09    129.2616
                       ...   
       2023-12-22    193.3533
       2023-12-26    192.8040
       2023-12-27    192.9038
       2023-12-28    193.3333
       2023-12-29    192.2846
Name: FIBBG000B9XRY4, Length: 250, dtype: float64


In [7]:
#calculating %returns
import pandas as pd 
returns=[0]
for i in range(1,len(apple_data)):
                returns.append((apple_data[i]-apple_data[i-1])/apple_data[i-1])
print(returns)

[0, 0.010314266324145852, -0.010604992131317355, 0.036794113714960296, 0.004089013727403191, 0.0044568533887869655, 0.02111174181304805, -0.0005988891134932025, 0.010119291106223888, 0.008755964202085723, -0.005369889069201903, 0.00044382520733554447, 0.019220420002873165, 0.02350051742181537, 0.010063019633698222, -0.004700576657758638, 0.014803568982417012, 0.013683980824941603, -0.02007811836982236, 0.009021016885186315, 0.00790062600702964, 0.037062448550339945, 0.02440007877669143, -0.017928852868838087, 0.01924492167255171, -0.017652987348497063, -0.006911315390070449, 0.002455949738454667, 0.018807228899620585, -0.004225226944218683, 0.01390345345325651, -0.010429415689947227, -0.007546409634393457, -0.02667982952429855, 0.002895897332077671, 0.003290588538705798, -0.01800523260039249, 0.008247042611205012, -0.003447863384955102, -0.014245641841594247, 0.004129003691058104, 0.035090016005754704, 0.018539807133285815, -0.014496779038162017, 0.008377593171130994, -0.01491466436579

In [8]:
#generating states based on %returns 1-bull 0-flat -1-bear
def state_assigner(value):
    if value>=0.01:
        return 1
    elif value>-0.01:
        return 0
    else:
        return -1
state_list=[state_assigner(val) for val in returns]
print(state_list)

[0, 1, -1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, -1, 0, 0, 1, 1, -1, 1, -1, 0, 0, 1, 0, 1, -1, 0, -1, 0, 0, -1, 0, 0, -1, 0, 1, 1, -1, 0, -1, -1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, -1, 0, 1, 0, 1, 0, 0, -1, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, -1, -1, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 1, -1, 1, 0, 1, 1, 0, 0, 0, -1, -1, 0, 0, -1, -1, 0, 0, 1, 0, -1, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0]


In [ ]:
#probability distribution
transition_states = {(prev, next1): 0 for prev in [-1, 0, 1] for next1 in [-1, 0, 1]}
for i in range(1, len(state_list)):
    prev = state_list[i-1]
    next1 = state_list[i]
    transition_states[(prev, next1)] += 1
    
transition_prob = {}
for (prev, next1), count in transition_states.items():
    total_occurrences = sum(value for key, value in transition_states.items() if key[0] == prev)
    probability = count / total_occurrences
    transition_prob[(prev, next1)] = probability

print(transition_prob.items())

In [14]:
#calculating portfolio_value and noting down the required indices(i.e days to execute buy)
#assumption: buy only if day d+1 is in bull state if  
def portfoliovalue(state_list):
    port_value=0
    transactions=[]
    for i in range(1,len(state_list)):
        prev_state=state_list[i-1]
        curr_state=state_list[i]
        if curr_state==1 and prev_state==0:
            port_value+=1
            transactions.append(i)
    return(port_value,transactions)
port_value,buy_indices=portfoliovalue(state_list)
print(port_value)
print(buy_indices)
        
    
    



41
[1, 6, 8, 12, 16, 21, 28, 30, 41, 50, 52, 59, 61, 69, 79, 85, 88, 94, 100, 103, 108, 110, 113, 117, 120, 123, 133, 142, 160, 164, 177, 187, 191, 207, 209, 212, 216, 218, 232, 234, 238]
